# Image simulation with dask and generat

The purpose here is to simulate images to identify the best methods for:

- Determining the FLAT image
- Segmenting cells from the background
- Computing the ratio
- Determine the minimal detectable gradient for a given error.

Since subtracting the correct background value is crucial for accurate ratio imaging, we will test the distribution of background values with probplot for normality.

External examples: https://cbia.fi.muni.cz/simulator/

In [ ]:
%load_ext autoreload
%autoreload 2

import dask.array as da
import dask_image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import skimage
import skimage.filters
import skimage.io
import tifffile as tff
import zarr
from dask_image import ndfilters, ndmorph
from scipy import ndimage

from nima import generat, nima, segmentation, utils

nrg = np.random.default_rng(1)

## FlatXY

In [ ]:
fp_flatxy = "/home/dati/dt-clop3/data/210920/flatxy.tf8"

In [ ]:
nima.read_tiffmd(fp_flatxy, channels=["G", "R", "C"])

In [ ]:
store = tff.imread(fp_flatxy, aszarr=True)

In [ ]:
zc1a = zarr.open(store)
zc1a.info

In [ ]:
dd = da.from_zarr(store)
dd

In [ ]:
img = dd[0, 0]
plt.imshow(img, vmax=60)

In [ ]:
from dask.diagnostics.progress import ProgressBar

with ProgressBar():
    bg_result = nima.bg(img.compute())

In [ ]:
bg_result.figures[1]

In [ ]:
im = img.compute()
bgmax = segmentation._bgmax(im, bins=25, densityplot=True)
bgmax

In [ ]:
bg_result = segmentation.bg_refine_iteratively(im, bgmax=bgmax, probplot=1)
bg, sd, iqr, figs = bg_result.bg, bg_result.sd, bg_result.iqr, bg_result.figures
bg, sd, iqr, figs

In [ ]:
pp = da.mean(
    dask_image.ndfilters.maximum_filter(dd[0:4000:20, 0], size=(100, 1, 1)), axis=0
)

In [ ]:
ppp = pp.compute()

plt.imshow(skimage.filters.gaussian(ppp, 100))

In [ ]:
m = img < skimage.filters.threshold_mean(img)
skimage.filters.threshold_mean((img * m).clip(np.min(img))).compute()

In [ ]:
plt.imshow(img < skimage.filters.threshold_triangle(img))

In [ ]:
def dabg(di):
    m = di < skimage.filters.threshold_mean(di)
    m1 = di < skimage.filters.threshold_mean((di * m).clip(np.min(di)))
    # m2 = ndmorph.binary_dilation(~m1)
    ndmorph.binary_dilation(~m1)
    return da.ma.masked_array(di, mask=~m1)


def bg(im):
    m = im < skimage.filters.threshold_mean(im)
    m1 = im < skimage.filters.threshold_mean((im * m).clip(np.min(im)))
    m2 = skimage.morphology.binary_dilation(~m1, footprint=np.ones([1, 1]))
    # m2 = im < skimage.filters.threshold_triangle(np.ma.masked_array(im, mask=~m))
    return np.ma.masked_array(im, mask=m2)

In [ ]:
tff.imshow(dabg(img).compute())

In [ ]:
flat = np.ma.mean(dabg(dd[333:500:1, 0]).compute(), axis=0)
skimage.io.imshow(flat)

### threshold mean clipping to min()

In [ ]:
tff.imshow(bg(dd[0, 0]))

In [ ]:
plt.hist(bg(img).ravel())

In [ ]:
[np.ma.median(bg(dd[i, 0])) for i in range(10)]

In [ ]:
segmentation.bg_refine_iteratively(img.compute(), bgmax=40)

In [ ]:
%%time
np.ma.mean(bg(img.compute()))

In [ ]:
%%time
utils.bg2(img.compute(), bgmax=60)

### masked array (ma)

In [ ]:
a = np.ma.masked_array([1, 4, 3], mask=[False, False, True])
b = np.ma.masked_array([10, 2, 6], mask=[False, True, False])

np.ma.median([a, b], axis=0)

In [ ]:
np.ma.median(np.ma.stack([a, b]), axis=0)

In [ ]:
f3 = img > skimage.filters.threshold_local(img.compute(), 601)

In [ ]:
f3

In [ ]:
img[~f3].mean().compute()

In [ ]:
m1 = np.ma.masked_greater(img, 15)

## generat

In [ ]:
image = "bias + noise + dark + flat * (sky + obj)"
image

- **bias**: generate a wave-like shape along x.
- **noise**: random number will do.
- **dark**: simply a scalar value.
- **flat**: generate some 2D parabolic shape.
- **obj**: circles-ellipsis. (MAYBE: like finite fractals to compare segmentation).
- **sky**: None | some blurred circle-ellipsoid coincident and not with some obj.

fg_prj := 

bg_prj := 

In [ ]:
X = Y = 128

plt.figure(figsize=(12, 2.8))

plt.subplot(1, 5, 1)
plt.title("BIAS")
bias = generat.gen_bias(Y, X)
skimage.io.imshow(bias)

plt.subplot(1, 5, 2)
plt.title("FLAT")
flat = generat.gen_flat(Y, X)
skimage.io.imshow(flat)

plt.subplot(1, 5, 3)
plt.title("Object")
single_obj = generat.gen_object(Y, X, max_radius=7)
skimage.io.imshow(single_obj)

plt.subplot(1, 5, 4)
plt.title("OBJS")
objects = generat.gen_objs(
    generat.ImageObjsParams(
        max_fluor=100, max_num_objects=13, max_radius=12, min_radius=6, ncols=Y, nrows=X
    )
)
skimage.io.imshow(objects)

plt.subplot(1, 5, 5)
plt.title("Frame")
frame = generat.gen_frame(objects, bias=bias, flat=flat, sky=17, noise_sd=3)
skimage.io.imshow(frame)

In [ ]:
objects = generat.gen_objs(
    generat.ImageObjsParams(
        max_fluor=100,
        max_num_objects=13,
        max_radius=12,
        min_radius=6,
        ncols=Y,
        nrows=X,
    )
)
frame = generat.gen_frame(objects, bias=np.ones((Y, X)), flat=None, sky=7, noise_sd=6)
segmentation.bg_refine_iteratively(frame, probplot=1)

In [ ]:
#         Method {'arcsinh', 'entropy', 'adaptive', 'li_adaptive', 'li_li'} used for the
bg_result = segmentation.bg(frame, bg_params=segmentation.BgParams(kind="adaptive"))
bg_result.bg

In [ ]:
segmentation.bg_refine_iteratively(
    frame, bgmax=segmentation._bgmax(frame, densityplot=True), probplot=True
)

In [ ]:
r2 = utils.bg2(frame, bgmax=30)

plt.plot(r2[3])
r2[:2]

In [ ]:
lim = np.arcsinh(frame)
plt.imshow(lim)
plt.colorbar()

In [ ]:
lim = ndimage.percentile_filter(lim, 80, size=10)
plt.imshow(lim)
plt.colorbar()

In [ ]:
bg_result = segmentation.bg_refine_iteratively(frame)
m0 = frame < bg_result.bg + 1 * bg_result.sd
plt.imshow(m0)

In [ ]:
p = segmentation.prob(frame, bg_result.bg, bg_result.sd)
plt.imshow(p)
plt.colorbar()
plt.figure()
plt.plot(p[100, :], "o")

In [ ]:
arcsinh_perc = 80
radius = 10
perc = 0.1

lim = np.arcsinh(frame)
lim = ndimage.percentile_filter(lim, arcsinh_perc, size=radius)
thr = (1 - perc) * lim.min() + perc * lim.max()
m = lim < thr
bgv = frame[m]

In [ ]:
plt.subplot(131)
plt.hist(p.ravel(), bins=99)
# plt.semilogy()
plt.subplot(132)
skimage.io.imshow(p)
plt.subplot(133)
skimage.io.imshow(p > 0.001)

In [ ]:
prob_frame = segmentation.geometric_mean_filter(p, 1.9)

In [ ]:
mask = prob_frame > 0.0029
segmentation.fit_gaussian(frame[mask]), scipy.stats.distributions.norm.fit(frame[mask])

In [ ]:
bg, sd = bg_result.bg, bg_result.sd
mgeo = segmentation.geometric_mean_filter(utils.prob(frame, bg, sd), 1) > 0.01
# mgeo = skimage.filters.median(utils.prob(frame, bg,  sd)) > 0.01

mgeo = (
    ndimage.percentile_filter(utils.prob(frame, bg, sd), percentile=1, size=2) > 0.005
)
# mgeo = ndimage.uniform_filter(utils.prob(frame, bg222,  sd222), size=1) > 0.005
# mgeo = ndimage.gaussian_filter(utils.prob(frame, bg222,  sd222), 0.25) > 0.005
skimage.io.imshow(mgeo)

In [ ]:
objs = generat.gen_objs(
    generat.ImageObjsParams(
        max_fluor=15,
        max_num_objects=20,
        max_radius=12,
        min_radius=6,
        ncols=264,
        nrows=64,
    )
)
frame = generat.gen_frame(objs, None, None, sky=10, noise_sd=6)

bgres = nima.bg(frame.astype("float"))
# plt.hist(bgs, bins=8)
bgres.bg

In [ ]:
segmentation.bg_refine_iteratively(frame, probplot=1)

simulation for:
- varying error
- varying number of cells
- varying intensity of cells
- in the absence of bias correction
- in the absence of flat correction

In [ ]:
objs_pars = generat.ImageObjsParams(max_num_objects=40, max_fluor=15, nrows=64)
objs_pars

In [ ]:
utils.bg2(frame)

In [ ]:
bg_arcsinh = []
bg_entropy = []
bg_adaptive = []
bg_liadaptive = []
bg_lili = []
bg_invyen = []
bg_ = []
bg2_ = []


def safe_call(func, *args, **kwargs):  # noqa: ANN002,ANN003
    """Wrap a function.

    This wrapper attempts to call the given function with the provided arguments.
    If the function call raises an exception, it returns NaN.
    """
    try:
        return func(*args, **kwargs)
    except ZeroDivisionError:
        print("Division by zero encountered.")
        return np.nan
    except Exception as e:  # noqa: BLE001
        print(f"An error occurred: {e}")
        return np.nan


objs_pars = generat.ImageObjsParams(max_num_objects=10, max_fluor=100, nrows=128)
noise_sd = 4

for _ in range(1):
    print(_)
    objs = generat.gen_objs(objs_pars)
    frame = generat.gen_frame(objs, None, None, sky=10, noise_sd=noise_sd)
    bg_arcsinh.append(
        safe_call(nima.bg, frame, segmentation.BgParams(kind="arcsinh")).bg
    )
    bg_entropy.append(
        safe_call(nima.bg, frame, segmentation.BgParams(kind="entropy")).bg
    )
    bg_adaptive.append(
        safe_call(nima.bg, frame, segmentation.BgParams(kind="adaptive")).bg
    )
    bg_liadaptive.append(
        safe_call(nima.bg, frame, segmentation.BgParams(kind="li_adaptive")).bg
    )
    bg_lili.append(safe_call(nima.bg, frame, segmentation.BgParams(kind="li_li")).bg)
    # bg_invyen.append(safe_call(nima.bg, frame, segmentation.BgParams(kind="inverse_yen")).bg)
    bg_.append(safe_call(segmentation.bg_refine_iteratively, frame).bg)
    bg2_.append(safe_call(utils.bg2, frame, bgmax=500)[0])

In [ ]:
def sim2df(num_of_repeats, noise_sd, objs_pars):
    bg_arcsinh = []
    bg_entropy = []
    bg_adaptive = []
    bg_liadaptive = []
    bg_lili = []
    bg_invyen = []
    bg_ = []
    bg2_ = []

    for _ in range(num_of_repeats):
        # print(_)
        objs = generat.gen_objs(objs_pars)
        frame = generat.gen_frame(objs, None, None, sky=10, noise_sd=noise_sd)
        bg_arcsinh.append(
            safe_call(nima.bg, frame, segmentation.BgParams(kind="arcsinh")).bg
        )
        bg_entropy.append(
            safe_call(nima.bg, frame, segmentation.BgParams(kind="entropy")).bg
        )
        bg_adaptive.append(
            safe_call(nima.bg, frame, segmentation.BgParams(kind="adaptive")).bg
        )
        bg_liadaptive.append(
            safe_call(nima.bg, frame, segmentation.BgParams(kind="li_adaptive")).bg
        )
        bg_lili.append(
            safe_call(nima.bg, frame, segmentation.BgParams(kind="li_li")).bg
        )
        bg_invyen.append(
            safe_call(nima.bg, frame, segmentation.BgParams(kind="inverse_yen")).bg
        )
        bg_.append(safe_call(segmentation.bg_refine_iteratively, frame).bg)
        bg2_.append(safe_call(utils.bg2, frame, bgmax=800)[0])

    df = pd.DataFrame(
        np.column_stack(
            (
                bg_arcsinh,
                bg_entropy,
                bg_adaptive,
                bg_liadaptive,
                bg_lili,
                bg_invyen,
                bg_,
                bg2_,
            )
        ),
        columns=[
            "arcsinh",
            "entropy",
            "adaptive",
            "li_adaptive",
            "li li",
            "inv_yen",
            "bg",
            "bg2",
        ],
    )
    df["sd"] = noise_sd
    return df

In [ ]:
objs_pars = generat.ImageObjsParams(max_num_objects=10, max_fluor=100, nrows=128)
# df_all

In [ ]:
df_all = pd.DataFrame()
for noise_sd in [1, 2, 3, 4]:
    df = sim2df(num_of_repeats=49, objs_pars=objs_pars, noise_sd=noise_sd)
    df_all = pd.concat([df_all, df])

f = df_all.boxplot(vert=False, showfliers=0, by="sd")
plt.xlim((5, 15))

In [ ]:
import seaborn as sns

In [ ]:
dfmelted = df_all.melt(ignore_index=1, id_vars="sd")
dfmelted.head()

In [ ]:
sns.barplot(dfmelted, x="variable", y="value", hue="sd", dodge=1, palette="Blues_d")
plt.ylim(4, 10.5)

In [ ]:
sns.stripplot(
    dfmelted,
    x="variable",
    y="value",
    hue="sd",
    dodge=1,
    jitter=0.2,
    alpha=0.6,
    palette="Reds_d",
)
sns.boxenplot(
    dfmelted, x="variable", y="value", hue="sd", dodge=1, alpha=0.6, palette="Reds_d"
)

plt.ylim(7.0, 11)

In [ ]:
skimage.io.imshow(frame)

In [ ]:
import seaborn as sns

sns.regplot(pd.DataFrame({"x": bg_arcsinh, "y": bg_}), x="x", y="y")

In [ ]:
# Generate a single small object
small_object = generat.gen_object(nrows=12, ncols=12, min_radius=3, max_radius=4)

# Plot the generated object
plt.imshow(small_object, cmap="gray")
plt.title("Single Small Object")
plt.xlabel("X")
plt.ylabel("Y")
plt.show()

In [ ]:
import scipy.signal

# Convolve the small object with the flat image
convolved_image = scipy.signal.convolve2d(flat, small_object, mode="same")

# Plot the convolved image
plt.imshow(convolved_image, cmap="gray")
plt.colorbar()
plt.title("Convolved Image")

In [ ]:
flat.shape[1] - small_object.shape[1]

In [ ]:
# Number of frames in the stack
num_frames = 10000

# Initialize an empty stack to store the frames
stack = np.zeros_like(flat)

# Iterate over each frame in the stack
for _ in range(num_frames):
    # Generate random coordinates for the position of the small object within the flat image
    x_pos = nrg.integers(0, flat.shape[1] - small_object.shape[1])
    y_pos = nrg.integers(0, flat.shape[0] - small_object.shape[0])

    # Add the small object to the flat image at the random position
    flat_image_with_object = flat.copy()
    flat_image_with_object[
        y_pos : y_pos + small_object.shape[0], x_pos : x_pos + small_object.shape[1]
    ] += small_object

    # Add the frame with the small object to the stack
    stack += flat_image_with_object

# Plot the summed stack
estimated = stack / stack.mean()
plt.imshow(estimated, cmap="gray")
plt.colorbar()
plt.title("Summed Stack with Small Object")
plt.show()
# plt.imshow(estimated - flat, cmap='gray')
skimage.io.imshow(ndimage.gaussian_filter(estimated, sigma=3) - flat)

In [ ]:
# Calculate the Fourier transform of the small object
fourier_transform_obj = np.fft.fft2(small_object)

# Calculate the magnitude spectrum of the Fourier transform
magnitude_spectrum = np.abs(np.fft.fftshift(fourier_transform_obj))

# Plot the magnitude spectrum
plt.imshow(magnitude_spectrum, cmap="gray")
plt.colorbar(label="Magnitude")
plt.title("Magnitude Spectrum of Fourier Transform")
plt.xlabel("Frequency (kx)")
plt.ylabel("Frequency (ky)")
plt.show()

In [ ]:
# Apply the convolution theorem
flat_fft = np.fft.fft2(stack)

# Calculate the Fourier transform of the small object
fourier_transform_obj = np.fft.fft2(small_object)

# Pad the small object to match the shape of flat
padded_obj = np.pad(
    small_object,
    (
        (0, flat.shape[0] - small_object.shape[0]),
        (0, flat.shape[1] - small_object.shape[1]),
    ),
    mode="constant",
)

# Calculate the Fourier transform of the padded small object
fourier_transform_padded_obj = np.fft.fft2(padded_obj)

# Calculate the Fourier transform of the flat image
flat_fft = np.fft.fft2(flat)

# Perform element-wise division
result_fft = np.fft.ifftshift(
    np.fft.ifft2(np.fft.fftshift(flat_fft / fourier_transform_padded_obj))
)
# result_fft = np.fft.ifftshift(np.fft.ifft2(np.fft.fftshift(flat_fft / fourier_transform_obj)))

# Take the real part to get rid of any numerical artifacts
result = np.real(result_fft)

# Plot the resulting flat image
plt.imshow(result, cmap="gray")
plt.colorbar(label="Intensity")
plt.title("Resulting Flat Image")
plt.xlabel("X")
plt.ylabel("Y")
plt.show()

In [ ]:
from nima import generat

flat = generat.gen_flat()
bias = 7 * np.ones((128, 128))

objs = generat.gen_objs(generat.ImageObjsParams(max_fluor=20, max_num_objects=80))
frame = generat.gen_frame(objs, bias=bias, flat=flat, noise_sd=2, sky=7)

# Plot the frame
plt.imshow(frame, cmap="viridis", origin="lower")
plt.colorbar(label="Intensity")
plt.title("Simulated Image Frame without Bias")
plt.xlabel("X")
plt.ylabel("Y")
plt.show()

In [ ]:
from tqdm import tqdm

# Generate a stack of frames
num_frames = 100
frame_stack = []
for _ in tqdm(range(num_frames), desc="Generating Frames"):
    objs = generat.gen_objs(generat.ImageObjsParams(max_fluor=20, max_num_objects=80))
    frame = generat.gen_frame(objs, bias=bias, flat=flat, noise_sd=2, sky=7)
    frame_stack.append(frame)

In [ ]:
from functools import partial

p999 = partial(np.percentile, q=99.7)
p999.__name__ = "percentile 99.9%"


def diff_plot(im, flat, title):
    f, axs = plt.subplots(1, 2)
    diff = im / im.mean() - flat
    skimage.io.imshow(diff, ax=axs[0])
    axs[1].hist(diff.ravel())
    f.suptitle(title)
    return diff.mean(), diff.std()


def prj_plot(t_prj, title, sigma=128 / 11):
    im = ndimage.gaussian_filter(t_prj, sigma=sigma)
    return diff_plot(im, flat, title)


def prj(stack, func):
    t_prj = func(stack, axis=0)
    return prj_plot(t_prj, func.__name__)


prj(frame_stack, np.max)
prj(frame_stack, p999)
prj(frame_stack, np.mean)
prj(frame_stack, np.median)
prj(frame_stack, np.min)

In [ ]:
objs = generat.gen_objs(generat.ImageObjsParams(max_fluor=20, max_num_objects=8))
frame = generat.gen_frame(objs, bias=bias, flat=flat)
plt.imshow(frame)

In [ ]:
bias = np.zeros((128, 128))
flat = np.ones((128, 128))

stack = np.stack(
    [
        generat.gen_frame(
            generat.gen_objs(generat.ImageObjsParams(max_fluor=20, max_num_objects=20)),
            None,
            None,
            noise_sd=8,
            sky=20,
        )
        for i in range(1000)
    ]
)

In [ ]:
stat_bg = []
stat_sd = []
for s in stack[:]:
    bg, sd = segmentation.iteratively_refine_background(s)[:2]
    stat_bg.append(bg)
    stat_sd.append(sd)

In [ ]:
plt.subplot(121)
plt.hist(stat_bg)
print(np.mean(stat_bg), np.std(stat_bg))
plt.subplot(122)
plt.hist(stat_sd)
np.mean(stat_sd), np.std(stat_sd)

In [ ]:
plt.imshow(stack[1])

## what is the best projection for flat calculation? 

In [ ]:
bias = generat.gen_bias()
flat = generat.gen_flat()
stack = np.stack(
    [
        generat.gen_frame(
            generat.gen_objs(generat.ImageObjsParams(max_fluor=20)),
            bias,
            flat,
            noise_sd=1,
            sky=2,
        )
        for i in range(1000)
    ]
)

In [ ]:
def splot(stack, num=4):
    _f, axs = plt.subplots(1, num)
    for i in range(num):
        axs[i].imshow(stack[nrg.integers(0, len(stack))])


splot(stack)

In [ ]:
prj(stack, np.max)

In [ ]:
prj(stack, np.mean)

In [ ]:
prj(stack, np.median)

In [ ]:
from functools import partial

p999 = partial(np.percentile, q=99.9)
p999.__name__ = "percentile 99.9%"

prj(stack, p999)

In [ ]:
im = np.mean(
    ndfilters.median_filter(
        da.from_array(stack[:100] - bias), size=(32, 16, 16)
    ).compute(),
    axis=0,
)
prj_plot(im, "dd", sigma=7)

#### Knowing the Bias.

In [ ]:
prj(stack - bias, p999)

In [ ]:
prj(stack - bias, np.mean)

### Using fg and bg masks?

And assuming we know the bias of the camera.

In [ ]:
def mask_plane(plane, bg_ave=2, bg_std=1.19, erf_pvalue=0.01):
    p = utils.prob(plane, bg_ave, bg_std)
    p = ndimage.median_filter(p, size=2)
    mask = p > erf_pvalue
    mask = skimage.morphology.remove_small_holes(mask)
    return np.ma.masked_array(plane, mask=~mask), np.ma.masked_array(plane, mask=mask)


plt.imshow(mask_plane(stack[113], *utils.bg(stack[113]))[0])

In [ ]:
bgs, fgs = list(
    zip(*[mask_plane(s - bias, *utils.bg(s - bias)) for s in stack], strict=False)
)

splot(bgs)

In [ ]:
t_prj = np.ma.mean(np.ma.stack(bgs), axis=0)
prj_plot(t_prj, "Bg mean", sigma=3)

In [ ]:
t_prj = np.ma.max(np.ma.stack(fgs), axis=0)
prj_plot(t_prj, "Fg max (bias known)", sigma=2)

In [ ]:
bgs, fgs = list(zip(*[mask_plane(s, *utils.bg(s)) for s in stack], strict=False))

bg_prj1 = np.ma.mean(np.ma.stack(bgs[:]), axis=0)
fg_prj1 = np.ma.mean(np.ma.stack(fgs[:]), axis=0)
im = fg_prj1 - bg_prj1
diff_plot(ndimage.gaussian_filter(im, 1), flat, "Bg mean - fg mean")

In [ ]:
bg_prj = np.ma.mean(bgs, axis=0)
fg_prj = np.ma.max(fgs, axis=0)
# im = ndimage.median_filter(bg_prj-fg_prj, size=60) #- 2 * flat
im = ndimage.gaussian_filter(bg_prj - fg_prj, sigma=14)  # - 2 * flat

diff_plot(im, flat, "m")

In [ ]:
t_prj = np.ma.max(fgs, axis=0)
prj_plot(t_prj, "Fg MAX", sigma=13)

In [ ]:
eflat = bg_prj - fg_prj
eflat /= eflat.mean()
eflat = ndimage.gaussian_filter(eflat, sigma=13)

diff_plot(eflat, flat, "eflat")

## When bias and flat are unknown...

- bias = bg_prj - sky * flat
- bias = fg_prj - flat

sky * flat - flat = bg_prj - fg_prj

In [ ]:
diff_plot((bg_prj1 - bias) / 2, flat, "")

In [ ]:
plt.imshow((im - bias) / (im - bias).mean() - flat)
plt.colorbar()

## cfr. nima.bg

In [ ]:
# r = nima.bg((stack[113] - bias) / flat)
r = nima.bg(stack[111])

In [ ]:
r[1].mean(), r[1].std()

In [ ]:
utils.bg(stack[111])

In [ ]:
bias.mean() + 2

## geometric mean

In [ ]:
vals = [0.8, 0.1, 0.3, 0.1, 0.8, 0.8, 0.8, 0.1, 0.8]

np.median(vals), scipy.stats.gmean(vals), np.mean(vals)

In [ ]:
p = vals[0]
p *= vals[1]
p *= vals[2]
p *= vals[3]
p *= vals[4]
p *= vals[5]
p *= vals[6]
p *= vals[7]
p *= vals[8]
np.power(p, 1 / 9)

In [ ]:
np.exp(np.log(np.array(vals)).sum() / 9)

In [ ]:
vv = np.array(vals).reshape(3, 3)
vv

In [ ]:
segmentation.geometric_mean_filter(vv, 1.0)

In [ ]:
kernel = skimage.morphology.disk(1.0).astype(float)
n = np.sum(kernel)  # Total weight, or number of ones in the kernel
print(n)

plt.imshow(kernel)

In [ ]:
(0.8 * 0.8 * 0.1 * 0.1 * 0.1) ** (1 / 5)